In [1]:
import os
import ssl
import certifi

ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

import asyncio
import re
from typing import Any

from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from pydantic import BaseModel, Field

import instructor
from instructor import Mode
from openai import OpenAI

C:\Users\pedro\OneDrive\Documentos\llama_pipeline\venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
client = instructor.patch(client) 

class CallReport(BaseModel):
    localizacao: str = Field(..., description="Local onde ocorre o incidente")
    nivel_urgencia: str = Field(..., description="Nível de urgência: baixo, médio ou alto")
    tipo_situacao: str = Field(..., description="Tipo de situação: incêndio, briga doméstica, acidente, etc.")
    pessoas_envolvidas: str = Field(..., description="Número ou descrição das pessoas envolvidas")
    resumo: str = Field(..., description="Resumo breve do que está acontecendo")

#Menos coerente, mais caótico e mais longo
transcript = """
Alô, polícia? Ai, meu Deus escuta, tem um um homem aqui na rua. 
Ele tá, ele tá batendo num carro, não, espera, agora ele tá gritando com ela, com a mulher. Acho que o carro é dele, ou dela, não sei.
É na Rua das Palmeiras, oitenta e cinco. Não, espera, eu tô no oitenta e cinco, a casa é a da esquina! Isso, a da esquina! 
Ele ele tá arroz a porta do carro, eu acho.
A mulher tá chorando muito. Ela, ela caiu? Não, levantou. Parece machucada, mas tá escuro, não vejo direito. 
Teve um barulho agora há pouco, um estalo, tipo tipo garrafa. Foi uma garrafa? Não sei. 
Ele jogou alguma coisa, mas não vi o que era. Ele tá ele não tá normal, moça, parece sei lá.
Por favor, tem como vir rápido? Ele Ah! Agora agora tem uma criança. Chorando. 
Antes não tinha. De onde, acho que é da casa. É o filho, eu não sei se é o filho dela.
Meu Deus, ele ele abriu a porta. Da casa. Ele tá entrando. Não, ele tá puxando ela pra dentro! Ele puxou ela! 
Ele farinha a porta fechou. Eu ouvi a tranca? Acho que ouvi a tranca. Ai, meu Deus.
Tem alguém lá dentro ela eu não ouço mais nada. Só a criança. Por favor, vem logo.
"""

report = client.chat.completions.create(
    model="llama3.1:8B",
    response_model=CallReport,
    messages=[
        {"role": "system", "content": "Você é um analista de chamadas de emergência. Sua tarefa é extrair informações essenciais da conversa."},
        {"role": "user", "content": f"""
Extraia as seguintes informações estruturadas desta transcrição de chamada de emergência, lembre de quebrar a linha:

- localizacao
- nivel_urgencia (baixo, medio, alto)
- tipo_situacao
- pessoas_envolvidas
- resumo

Transcrição:
{transcript}
""",},
    ],
    temperature=0,
    timeout=2400,
)

print(report)


2025-10-20 03:21:53,825 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


localizacao='Rua das Palmeiras, oitenta e cinco' nivel_urgencia='alto' tipo_situacao='briga doméstica' pessoas_envolvidas='homem, mulher, criança' resumo='Homem agredindo mulher e criança em Rua das Palmeiras, oitenta e cinco.'
